In [1]:
!nvidia-smi

Fri May 26 09:18:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   35C    P0    46W / 163W |   8324MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# sync python module
%load_ext autoreload
%autoreload 2

In [6]:
import os

workspace_dir = '/nfs/Workspace/CardiacSeg'
model_name = 'unetcnx_a1' #'unetcst_a9' #'unetcnx_x3_2_2' #'unetcnx_x0' #'unetcnx_x3_2_2' #'unetcnx_x3_2_2' #'unetcnx_x3_2_2' #'unetcnx_x0'
data_name = 'chgh'
sub_data_dir_name = 'dataset_2'
exp_name = 'exp_b7_9_ds1_tn1' #'exp_b7_9_x3_2_2_a5_p1' #'t_3_f0' #'exp_b7_3_x3_2_2_10'#'exp_2_2_x3_2_2_tn2' #'exp_2_2_x3_2_2_tn1' #'t_1' #'exp_2_2_1_x0' #'exp_2_2_x0_t6'
data_dict_file_name = 'exp_b7_9.json'

tune_mode = 'test' #'lrschedule' #'train'


root_exp_dir = os.path.join(
    workspace_dir, 
    'exps',
    'exps',
    model_name,
    data_name,
    'tune_results'
)

root_data_dir = os.path.join(
    workspace_dir, 
    'dataset',
    data_name
)

data_dir = os.path.join(root_data_dir, sub_data_dir_name)
data_dicts_json = os.path.join(workspace_dir, 'exps', 'data_dicts', data_name, data_dict_file_name)

exp_dir = '/nfs/Workspace/CardiacSeg/exps/exps/unetcnx_a1/chgh/tune_results/exp_b7_9_ds1_tn1/main_5cbaa_00000_0_optim=lr_0_0007_weight_decay_0_0005_2023-05-17_05-19-13/'
model_dir = os.path.join(exp_dir, 'models')
log_dir = os.path.join(exp_dir, 'logs')
eval_dir = os.path.join(exp_dir, 'eda_evals')

best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')

# test train data or val data for eda (train, val), default is test data
# val dice value diff training, because infer not include bg
eda_test_data = 'test' # val, test

os.makedirs(root_exp_dir, exist_ok=True)

%cd {root_exp_dir}/../

/nfs/Workspace/CardiacSeg/exps/exps/unetcnx_a1/chgh


In [7]:
!PYTHONPATH=/nfs/Workspace/CardiacSeg /opt/conda/bin/python /nfs/Workspace/CardiacSeg/expers/train.py \
--tune_mode={tune_mode} \
--exp_name={exp_name} \
--data_name={data_name} \
--data_dir={data_dir} \
--root_exp_dir={root_exp_dir} \
--model_name={model_name}\
--model_dir={model_dir} \
--log_dir={log_dir} \
--eval_dir={eval_dir} \
--start_epoch=0 \
--val_every=20 \
--max_early_stop_count=20 \
--max_epoch=8000  \
--data_dicts_json={data_dicts_json} \
--pin_memory \
--out_channels=2 \
--patch_size=4 \
--feature_size=48 \
--drop_rate=0.1 \
--depths 3 3 9 3 \
--kernel_size 7 \
--exp_rate 4 \
--norm_name='layer' \
--a_min=-42 \
--a_max=423 \
--space_x=0.7 \
--space_y=0.7 \
--space_z=1.0 \
--roi_x=128 \
--roi_y=128 \
--roi_z=128 \
--optim='AdamW' \
--lr=5e-4 \
--weight_decay=5e-4 \
--checkpoint={best_checkpoint} \
--use_init_weights \
--deep_sup \
--eda_test_data=eda_test_data \
--infer_post_process \
--test_mode

cuda is available
model: unetcnx_a1
patch size: 4
ker size: 7
exp rate: 4
feature sizes: [48, 96, 192, 384]
depths: [3, 3, 9, 3]
drop rate: 0.1
use init weights: True
is conv stem: False
use init weights
use deep sup
optimzer: AdamW
{'lr': 0.0005, 'weight_decay': 0.0005}
=> loaded checkpoint '/nfs/Workspace/CardiacSeg/exps/exps/unetcnx_a1/chgh/tune_results/exp_b7_9_ds1_tn1/main_5cbaa_00000_0_optim=lr_0_0007_weight_decay_0_0005_2023-05-17_05-19-13/models/best_model.pth' (epoch 521) (bestacc 0.8913398385047913) (early stop count 0)
load json from /nfs/Workspace/CardiacSeg/exps/data_dicts/chgh/exp_b7_9.json
train files (9): ['pid_08', 'pid_27', 'pid_30', 'pid_52', 'pid_56', 'pid_57', 'pid_110', 'pid_1002', 'pid_1003']
val files (3): ['pid_08_1', 'pid_107', 'pid_108']
test files (3): ['pid_02', 'pid_106', 'pid_1000']
infer data: {'image': '/nfs/Workspace/CardiacSeg/dataset/chgh/dataset_2/pid_02/pid_02.nii.gz', 'label': '/nfs/Workspace/CardiacSeg/dataset/chgh/dataset_2/pid_02/pid_02_gt.nii.